# Pods Experiments

##### NOT USED FOR GPU-AWARE YET (Friday) Feel free to add experiments here though, giving some description of what you're doing. If you want of course, this is all brownie points for an invisible audience.

## Create fake testing data. Create csvs with 10000, 200000, and 500000 rows.

In [186]:
## Creating fake data to use in Neo4j and Postgres
number_of_rows = 10000

persons = []
for i in range(number_of_rows):
    person = {"name": fake.name(),
              "email": fake.email(),
              "phone number": fake.phone_number(),
              "address": fake.address(),
              "description": fake.text(),
              "company": fake.company()}
    persons.append(person)
    if i % 1000 == 0:
        print(i, end="\r", flush=True)

# Define the CSV file name
csv_file_name = f"syntheticdata{number_of_rows}.csv"

# Write the list of dictionaries to the CSV file
with open(csv_file_name, mode='w', newline='') as csv_file:
    fieldnames = persons[0].keys()
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for row in persons:
        writer.writerow(row)

In [ ]:
# We then upload these csvs to a single Pods service Volume
volume_id = "myvolume"
for csv_file in ["syntheticdata10000.csv", "syntheticdata200000.csv", "syntheticdata500000.csv"]:
with open(csv_file, "rb") as file:
    t.pods.upload_to_volume(volume_id=volume_id, path=csv_file, file=file)

# Neo4j Testing
For these tests we will create a Neo4j instance in the Pods service, in AuraDB, and locally with Docker. We will then import our synthetic data to each of these three instances. We'll then run our experiments by changing the urls and executing code in the "Neo4j Testing Function" code block. We have three sets of synthetic data, so we'll delete and recreate three times, each with a different set of data.

## Neo4j Pods
You'll need to create the Pod, get it's credentials from the API and then load the CSV into the database. 

In [273]:
# Import libs, setup credentials, and neo4j driver
from neo4j import GraphDatabase
import timeit

url = "bolt+s://neo4jpaper.pods.icicle.develop.tapis.io:443"
user = "neo4jpaper"
passw = "password"
neo = GraphDatabase.driver(url, auth = (user, passw), max_connection_lifetime=30)

In [288]:
# Load CSV
query = f"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///syntheticdata10000.csv' AS row
CREATE (w:Worker {{
    name: row.name,
    email: row.email,
    phone_number: row.`phone number`,
    address: row.address,
    description: row.description,
    company: row.company
}})
"""
with neo.session() as session:
    session.run(query)

## Neo4j AuraDB
You'll need to create an AuraDB database, get it's credentials from the website, and then load the CSV into the database.

AuraDB site: https://neo4j.com/cloud/platform/aura-graph-database/  

With AuraDB you can upload csvs with the Neo4j Data Importer, located here: https://data-importer.neo4j.io/  

AuraDB Free max node count is 200000, so we only test up to 200000 nodes with it.

In [224]:
# Import libs, setup credentials, and neo4j driver
from neo4j import GraphDatabase
import timeit

url = "neo4j+s://neo4jpaper.databases.neo4j.io"
user = "neo4jpaper"
passw = "password"
neo = GraphDatabase.driver(url, auth = (user, passw), max_connection_lifetime=30)

## Neo4j Local
Create local instance with docker-compose.yml. Ensure csv is in correct file path for volume mount to work. Then connect and load the CSV.

In [263]:
# Import libs, setup credentials, and neo4j driver
from neo4j import GraphDatabase
import timeit

url = "bolt://localhost:7811"
user = "neo4j"
passw = "newPassword"
neo = GraphDatabase.driver(url, auth = (user, passw), max_connection_lifetime=30)

In [262]:
# Load CSV
query = f"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM 'file:///syntheticdata200000.csv' AS row
CREATE (w:Worker {{
    name: row.name,
    email: row.email,
    phone_number: row.`phone number`,
    address: row.address,
    description: row.description,
    company: row.company
}})
"""
with neo.session() as session:
    session.run(query)

## Neo4j Testing Function
This is how we'll run our test for Neo4j. You should, above, define the neo4j credentials you want to use, url, user, passw, and neo. 

In [292]:
import time
print(f"Neo4J url: {url}")
for iteration in range(60):
    start_t = timeit.default_timer()
    with neo.session() as session:
        result = session.run("MATCH (n) RETURN n")
        node_count = len(result.values())
    end_t = timeit.default_timer()
    time.sleep(2)
    print(f"iteration: {iteration} number_of_items: {node_count} time_total: {end_t-start_t}")

Neo4J url: bolt+s://neo4jpaper.pods.icicle.develop.tapis.io:443
iteration: 0 number_of_nodes: 10000 time_total: 2.3528107460588217
iteration: 1 number_of_nodes: 10000 time_total: 1.8315149834379554
iteration: 2 number_of_nodes: 10000 time_total: 2.4489784091711044
iteration: 3 number_of_nodes: 10000 time_total: 2.3645988339558244
iteration: 4 number_of_nodes: 10000 time_total: 1.8414975395426154
iteration: 5 number_of_nodes: 10000 time_total: 1.8316038064658642
iteration: 6 number_of_nodes: 10000 time_total: 1.8185261134058237
iteration: 7 number_of_nodes: 10000 time_total: 1.8369185803458095
iteration: 8 number_of_nodes: 10000 time_total: 2.400876132771373
iteration: 9 number_of_nodes: 10000 time_total: 1.825438848696649
iteration: 10 number_of_nodes: 10000 time_total: 2.3966366592794657
iteration: 11 number_of_nodes: 10000 time_total: 1.8682471001520753
iteration: 12 number_of_nodes: 10000 time_total: 1.8339237933978438
iteration: 13 number_of_nodes: 10000 time_total: 1.8139731381088

### Neo4j Misc

In [283]:
# If you're looking for the amount of nodes in the database/you want to test the neo4j connection.
# Match and return # of nodes
with neo.session() as session:
    print(f"Neo4J url: {url}")
    result = session.run("MATCH (n) RETURN n")
    print(f"Number of nodes in the database: {len(result.values())}")

Neo4J url: bolt+s://neo4jpaper.pods.icicle.develop.tapis.io:443
Number of nodes in the database: 200000


# Postgres Testing
For these tests we will create a Postgres instance in the Pods service, in Google Cloud Postgres, and locally with Docker. We will then import our synthetic data to each of these three instances. We'll then run our experiments by changing the urls and executing code in the "Postgres Testing Function" code block. We have three sets of synthetic data, so we'll delete and recreate three times, each with a different set of data.

## Postgres with Pods Service
You'll need to create the Pod, get it's credentials from the API and then load the CSV into the database.

In [37]:
# Import libs, setup credentials, and postgres connection
import psycopg2
import csv


url = "postgrespaper.pods.icicle.develop.tapis.io"
port = "443"
user = "postgrespaper"
passw = "password"

# Connect to the PostgreSQL database
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

In [41]:
# First create workers table
# Connect to the PostgreSQL database
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

# Create the 'workers' table
create_table_query = '''
CREATE TABLE workers (
    name VARCHAR(255),
    email VARCHAR(255),
    phone_number VARCHAR(255),
    address VARCHAR(255),
    description TEXT,
    company VARCHAR(255)
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table 'workers' created.")
except psycopg2.errors.DuplicateTable:
    print("Table 'workers' already exists")
    pass

# Close the database connection
cursor.close()
connection.close()

Table 'workers' created.


In [43]:
# Now add data from csv to workers table
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

cursor.execute("COPY workers (name, email, phone_number, address, description, company) FROM '/tapisvolume/syntheticdata10000.csv' DELIMITER ',' CSV HEADER;")

# Commit the changes
connection.commit()

# Close the database connection
cursor.close()
connection.close()

print("CSV data loaded into the 'workers' table.")

CSV data loaded into the 'workers' table.


## Postgres on localhost
You'll need to create the Pod, get it's credentials from the API and then load the CSV into the database.

In [362]:
# Import libs, setup credentials, and postgres connection
import psycopg2
import csv

url = "localhost"
port = "7812"
user = "postgres"
passw = "newPassword"

# Connect to the PostgreSQL database
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

In [9]:
# First create workers table
# Connect to the PostgreSQL database
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

# Create the 'workers' table
create_table_query = '''
CREATE TABLE workers (
    name VARCHAR(255),
    email VARCHAR(255),
    phone_number VARCHAR(255),
    address VARCHAR(255),
    description TEXT,
    company VARCHAR(255)
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table 'workers' created.")
except psycopg2.errors.DuplicateTable:
    print("Table 'workers' already exists")
    pass

# Close the database connection
cursor.close()
connection.close()

Table 'workers' created.


In [364]:
# Now add data from csv to workers table
# Replace this with the path to your CSV file
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

cursor.execute("COPY workers (name, email, phone_number, address, description, company) FROM '/syntheticdata500000.csv' DELIMITER ',' CSV HEADER;")

# Commit the changes
connection.commit()

# Close the database connection
cursor.close()
connection.close()

print("CSV data loaded into the 'workers' table.")

CSV data loaded into the 'workers' table.


## Postgres on Google Cloud
You'll need to create the DB, get it's credentials from the API and then load the CSV into the database.

Link: https://cloud.google.com/sql/docs/postgres

To note, you'll also need to add your local IP to Google Cloud's IP allowlist. This is a guide: https://medium.com/google-cloud/cloud-sql-public-ip-allowlist-e2e375b4432d

In [22]:
import psycopg2

## Google Cloud Postgres
user = "postgres"
passw = "password"
url = "22.322.322.22"

connection = psycopg2.connect(host=url, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

In [33]:
# First create workers table
# Connect to the PostgreSQL database
connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()

# Create the 'workers' table
create_table_query = '''
CREATE TABLE workers (
    name VARCHAR(255),
    email VARCHAR(255),
    phone_number VARCHAR(255),
    address VARCHAR(255),
    description TEXT,
    company VARCHAR(255)
);
'''

try:
    cursor.execute(create_table_query)
    connection.commit()
    print("Table 'workers' created.")
except psycopg2.errors.DuplicateTable:
    print("Table 'workers' already exists")
    pass

# Close the database connection
cursor.close()
connection.close()

Table 'workers' created.


## Postgres Testing Function

In [ ]:
import time
import timeit
print(f"Postgres url: {url}:{port}")
for iteration in range(60):
    
    # Create connection and get cursor
    connection = psycopg2.connect(host=url, port=port, dbname="postgres", user=user, password=passw)
    cursor = connection.cursor()
    
    start_t = timeit.default_timer()

    select_query = "SELECT * FROM workers;"
    cursor.execute(select_query)
    row_count = len(cursor.fetchall())

    end_t = timeit.default_timer()
    
    # Close the database connection
    cursor.close()
    connection.close()
    
    print(f"iteration: {iteration} number_of_items: {row_count} time_total: {end_t-start_t}")
    time.sleep(1)

In [40]:
# Delete current workers table
connection = psycopg2.connect(host=url, port=443, dbname="postgres", user=user, password=passw)
cursor = connection.cursor()
table_name='workers'
cursor.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE")
connection.commit()
# Close the database connection
cursor.close()
connection.close()

# FastAPI Testing
Create local instance with docker-compose.yml. Get url.  
Create pod instance with API, get url.

## FastAPI Testing Function

In [ ]:
import requests as r
import time
import timeit

for iteration in range(60):
    start_t = timeit.default_timer()
    result = r.get('https://fastapipaper.pods.icicle.develop.tapis.io')
    end_t = timeit.default_timer()
    time.sleep(.2)
    #print(f"iteration: {iteration} time_total: {end_t-start_t}")
    print(end_t-start_t)
    #print(result.json())

## Log Parsing

In [35]:
car = """iteration: 0 number_of_items: 500001 time_total: 4.06858329847455
iteration: 1 number_of_items: 500001 time_total: 3.8626063698902726
iteration: 2 number_of_items: 500001 time_total: 5.630214522592723
iteration: 3 number_of_items: 500001 time_total: 4.610894144512713
iteration: 4 number_of_items: 500001 time_total: 5.150433995760977
iteration: 5 number_of_items: 500001 time_total: 3.903535292483866
iteration: 6 number_of_items: 500001 time_total: 3.9847047552466393
iteration: 7 number_of_items: 500001 time_total: 5.040335590019822
iteration: 8 number_of_items: 500001 time_total: 3.870199089869857
iteration: 9 number_of_items: 500001 time_total: 4.046817721799016
iteration: 10 number_of_items: 500001 time_total: 3.8450532825663686
iteration: 11 number_of_items: 500001 time_total: 3.918391726911068
iteration: 12 number_of_items: 500001 time_total: 3.9219076046720147
iteration: 13 number_of_items: 500001 time_total: 4.878413186408579
iteration: 14 number_of_items: 500001 time_total: 4.3378392942249775
iteration: 15 number_of_items: 500001 time_total: 3.904729143716395
iteration: 16 number_of_items: 500001 time_total: 5.43999008834362
iteration: 17 number_of_items: 500001 time_total: 5.187447725795209
iteration: 18 number_of_items: 500001 time_total: 3.9014492658898234
iteration: 19 number_of_items: 500001 time_total: 4.013528158888221
iteration: 20 number_of_items: 500001 time_total: 4.975224238820374
iteration: 21 number_of_items: 500001 time_total: 5.268680344335735
iteration: 22 number_of_items: 500001 time_total: 4.003169623203576
iteration: 23 number_of_items: 500001 time_total: 5.035185583867133
iteration: 24 number_of_items: 500001 time_total: 3.926520451903343
iteration: 25 number_of_items: 500001 time_total: 3.8629355849698186
iteration: 26 number_of_items: 500001 time_total: 3.9954193020239472
iteration: 27 number_of_items: 500001 time_total: 4.834351510740817
iteration: 28 number_of_items: 500001 time_total: 4.973814412020147
iteration: 29 number_of_items: 500001 time_total: 6.296275618486106
iteration: 30 number_of_items: 500001 time_total: 3.9011220717802644
iteration: 31 number_of_items: 500001 time_total: 4.441186179406941
iteration: 32 number_of_items: 500001 time_total: 5.158780609257519
iteration: 33 number_of_items: 500001 time_total: 4.009407182224095
iteration: 34 number_of_items: 500001 time_total: 3.8955306131392717
iteration: 35 number_of_items: 500001 time_total: 4.382479537278414
iteration: 36 number_of_items: 500001 time_total: 3.915195510722697
iteration: 37 number_of_items: 500001 time_total: 5.811555355787277
iteration: 38 number_of_items: 500001 time_total: 4.3888355335220695
iteration: 39 number_of_items: 500001 time_total: 5.213144165463746
iteration: 40 number_of_items: 500001 time_total: 3.871880735270679
iteration: 41 number_of_items: 500001 time_total: 5.155211192555726
iteration: 42 number_of_items: 500001 time_total: 5.100531295873225
iteration: 43 number_of_items: 500001 time_total: 4.499079331755638
iteration: 44 number_of_items: 500001 time_total: 4.412205909378827
iteration: 45 number_of_items: 500001 time_total: 4.178113849833608
iteration: 46 number_of_items: 500001 time_total: 3.8537090085446835
iteration: 47 number_of_items: 500001 time_total: 6.7891157902777195
iteration: 48 number_of_items: 500001 time_total: 5.5942978309467435
iteration: 49 number_of_items: 500001 time_total: 4.90167126711458
iteration: 50 number_of_items: 500001 time_total: 4.951330647803843
iteration: 51 number_of_items: 500001 time_total: 4.419565484859049
iteration: 52 number_of_items: 500001 time_total: 3.86984998639673
iteration: 53 number_of_items: 500001 time_total: 5.556173398159444
iteration: 54 number_of_items: 500001 time_total: 4.682019389234483
iteration: 55 number_of_items: 500001 time_total: 4.023812619037926
iteration: 56 number_of_items: 500001 time_total: 3.940865189768374
iteration: 57 number_of_items: 500001 time_total: 5.182402773760259
iteration: 58 number_of_items: 500001 time_total: 3.9073168393224478
iteration: 59 number_of_items: 500001 time_total: 3.847339735366404"""

In [36]:
for line in car.split('\n'):
    iteration, time = line.replace('iteration: ', '').split("number_of_items: 500001 time_total: ")
    iteration = int(iteration) + 1
    #print(iteration)
    print(time)

4.06858329847455
3.8626063698902726
5.630214522592723
4.610894144512713
5.150433995760977
3.903535292483866
3.9847047552466393
5.040335590019822
3.870199089869857
4.046817721799016
3.8450532825663686
3.918391726911068
3.9219076046720147
4.878413186408579
4.3378392942249775
3.904729143716395
5.43999008834362
5.187447725795209
3.9014492658898234
4.013528158888221
4.975224238820374
5.268680344335735
4.003169623203576
5.035185583867133
3.926520451903343
3.8629355849698186
3.9954193020239472
4.834351510740817
4.973814412020147
6.296275618486106
3.9011220717802644
4.441186179406941
5.158780609257519
4.009407182224095
3.8955306131392717
4.382479537278414
3.915195510722697
5.811555355787277
4.3888355335220695
5.213144165463746
3.871880735270679
5.155211192555726
5.100531295873225
4.499079331755638
4.412205909378827
4.178113849833608
3.8537090085446835
6.7891157902777195
5.5942978309467435
4.90167126711458
4.951330647803843
4.419565484859049
3.86984998639673
5.556173398159444
4.682019389234483
